In [1]:
import json
import numpy as np
import nltk
from pprint import pprint
from nltk.corpus import wordnet
from string import punctuation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import gensim 
from gensim.models import Word2Vec, KeyedVectors
import os
import re
from prettytable import PrettyTable
import pandas as pd

nltk.download("wordnet")
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()


[nltk_data] Downloading package wordnet to /home/vatsal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vatsal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def entry(line): 
    w, c = line.split("\t", 2)
    return (w, int(c))

dict_path = "./dict.txt"
dictionary = dict(entry(line) for line in open(dict_path))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))
cleanup = re.compile(r'[^a-z0-9]')

def word_prob(word): 
    return dictionary.get(word, 0) / total

def segment(text): 
    text = re.sub(cleanup, ' ', text)
    probs, lasts = [1.0], [0]
    for i in range(1,len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while i > 0:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words

In [3]:
glob_corpora = []
from glove import Corpus, Glove
corpus = Corpus() 

In [4]:
top100 = {}
top100['travel'] = ['#travel', '#wanderlust', '#nature', '#travelling', '#traveling', '#traveller', '#photography', '#traveler', '#trip', '#travels', '#vacation', '#love', '#travelers', '#adventure', '#tourist', '#landscape', '#travellers', '#explore', '#holiday', '#beautiful', '#tourism', '#hiking', '#beach', '#photo', '#sunset', '#photographer', '#mountains', '#globetrotter', '#summer', '#art', '#sky', '#treking', '#europe', '#view', '#architecture', '#sea', '#happy', '#fun', '#city', '#sun', '#lifestyle', '#amazing', '#wanderer', '#italy', '#follow', '#backpacking', '#life', '#visiting', '#fashion', '#autumn', '#explorer', '#ocean', '#outdoors', '#india', '#world', '#mountain', '#beauty', '#spain', '#backpacker', '#style', '#like', '#france', '#exploring', '#trekking', '#clouds', '#asia', '#me', '#friends', '#usa', '#canon', '#happiness', '#blogger', '#holidays', '#ig', '#sunrise', '#smile', '#germany', '#girl', '#island', '#wander', '#paradise', '#turkey', '#discover', '#italia', '#voyage', '#flowers', '#landscapes', '#sightseeing', '#outdoor', '#history', '#indonesia', '#cute', '#forest', '#paris', '#food', '#australia', '#bali', '#pic', '#beaches', '#inspiration']
top100['food'] = ['#foodie', '#food', '#delicious', '#yummy', '#foodies', '#foods', '#eat', '#breakfast', '#dinner', '#tasty', '#cooking', '#lunch', '#homemade', '#dessert', '#love', '#eating', '#healthy', '#sweet', '#restaurant', '#photography', '#hungry', '#chef', '#blogger', '#travel', '#chocolate', '#baking', '#follow', '#cake', '#vegan', '#fresh', '#chicken', '#like', '#cook', '#amazing', '#blog', '#happy', '#favorite', '#brunch', '#coffee', '#weekend', '#fit', '#vegetarian', '#pasta', '#beautiful', '#pastry', '#fitness', '#gourmet', '#morning', '#desserts', '#seafood', '#eater', '#lifestyle', '#sweets', '#cafe', '#recipes', '#icecream', '#pizza', '#italy', '#culinary', '#candy', '#cheese', '#photographer', '#photo', '#cakes', '#recipe', '#noodles', '#diet', '#eats', '#rice', '#friends', '#cuisine', '#porridge', '#salad', '#nutrition', '#india', '#indonesia', '#bread', '#mornings', '#drinks', '#art', '#life', '#fun', '#gastronomy', '#sunday', '#cookies', '#kitchen', '#gym', '#me', '#beef', '#sushi', '#cupcakes', '#bake', '#spicy', '#saturday', '#fish', '#catering', '#burger', '#snack', '#music', '#delhi']
top100['baby'] = ['#baby', '#kids', '#love', '#babies', '#cute', '#family', '#mom', '#fashion', '#newborn', '#children', '#happy', '#beautiful', '#photography', '#girl', '#babys', '#adorable', '#sweet', '#motherhood', '#pregnancy', '#handmade', '#funny', '#lovely', '#smile', '#mommy', '#life', '#daughter', '#style', '#follow', '#sweetheart', '#like', '#pregnant', '#beauty', '#precious', '#kid', '#child', '#fun', '#girls', '#mummy', '#tiny', '#twins', '#amazing', '#toddler', '#model', '#photo', '#mama', '#toys', '#boy', '#sleep', '#matching', '#autumn', '#enjoy', '#friends', '#pretty', '#art', '#parenting', '#live', '#mother', '#me', '#amor', '#photographer', '#princess', '#fairy', '#sale', '#happiness', '#boys', '#boutique', '#lifestyle', '#portrait', '#fall', '#son', '#pink', '#nature', '#childhood', '#cuteness', '#maternity', '#cool', '#travel', '#angel', '#best', '#halloween', '#play', '#weekend', '#october', '#dad', '#innocent', '#brand', '#makeup', '#home', '#parenthood', '#nice', '#little', '#sunday', '#flowers', '#mum', '#canon', '#infant', '#party', '#wedding', '#summer', '#hot']
top100['jewellery'] = ['#jewellery', '#jewelry', '#fashion', '#accessories', '#earrings', '#necklace', '#style', '#jewels', '#handmade', '#beautiful', '#gemstone', '#trendy', '#gold', '#jewel', '#bracelet', '#rings', '#love', '#gems', '#silver', '#bracelets', '#design', '#ring', '#gemstones', '#crystals', '#shopping', '#stylish', '#art', '#pendant', '#diamond', '#cute', '#luxury', '#necklaces', '#bling', '#india', '#watches', '#handcrafted', '#designer', '#bijoux', '#trending', '#earring', '#bangles', '#bride', '#wedding', '#beauty', '#stone', '#tourmaline', '#charm', '#traditional', '#hyderabad', '#wholesaler', '#diamonds', '#bridal', '#photography', '#accessory', '#cabochons', '#tourmalines', '#stones', '#gem', '#goldplated', '#semiprecious', '#mumbai', '#silversmith', '#pendents', '#k', '#indian', '#sapphire', '#chennai', '#jewellers', '#girls', '#sale', '#jeweller', '#delhi', '#follow', '#fashionable', '#gift', '#bangalore', '#women', '#london', '#unique', '#ethnic', '#rawalpindi', '#crystal', '#vintage', '#oxidised', '#emerald', '#dubai', '#tucson', '#peridot', '#beads', '#makeup', '#saree', '#kunzite', '#chic', '#firecracker', '#pearl', '#lahore', '#choker', '#islamabad', '#wholesale', '#combo']
# top100['selfie'] = ['#me', '#love', '#follow', '#smile', '#photography', '#style', '#beautiful', '#fashion', '#happy', '#cute', '#fun', '#girl', '#life', '#portrait', '#like', '#pretty', '#photo', '#handsome', '#travel', '#nature', '#eyes', '#face', '#hair', '#summer', '#art', '#friends', '#amazing', '#model', '#food', '#beauty', '#pic', '#daily', '#girls', '#lifestyle', '#look', '#makeup', '#swag', '#photographer', '#fitness', '#autumn', '#cool', '#family', '#sky', '#picture', '#likes', '#sun', '#mood', '#music', '#followers', '#outfit', '#boy', '#colorful', '#self', '#motivation', '#lovely', '#gay', '#pose', '#landscape', '#black', '#inspiration', '#happiness', '#halloween', '#artist', '#gym', '#liker', '#pink', '#portraits', '#blogger', '#fit', '#sunset', '#wanderlust', '#design', '#funny', '#italy', '#traveling', '#hot', '#awesome', '#holiday', '#following', '#photos', '#india', '#night', '#blue', '#weekend', '#view', '#october', '#party', '#trip', '#red', '#tattoo', '#man', '#looks', '#stylish', '#sea', '#nice', '#good', '#beard', '#beach', '#woman', '#aesthetic']
top100['pets'] = ['#pet', '#pets', '#dog', '#cute', '#animals', '#love', '#dogs', '#cat', '#puppy', '#animal', '#cats', '#kitten', '#nature', '#kitty', '#meow', '#adorable', '#puppies', '#kittens', '#doggy', '#doggo', '#pup', '#photography', '#happy', '#funny', '#hound', '#eyes', '#beautiful', '#bunny', '#poodle', '#chihuahua', '#follow', '#rabbit', '#fluffy', '#paws', '#family', '#photo', '#lovely', '#chat', '#furry', '#daily', '#pomeranian', '#labrador', '#fun', '#autumn', '#sweet', '#maltese', '#smile', '#life', '#art', '#bunnies', '#hamster', '#baby', '#bird', '#me', '#world', '#bulldog', '#like', '#sleeping', '#tuesday', '#summer', '#parrots', '#husky', '#amor', '#rabbits', '#tot', '#zoo', '#doggie', '#meme', '#pug', '#istanbul', '#wildlife', '#beauty', '#adopt', '#girl', '#meowed', '#birds', '#purr', '#rescue', '#cavy', '#japan', '#doggies', '#halloween', '#photographer', '#woof', '#playtime', '#nice', '#portrait', '#relax', '#parakeet', '#singapore', '#parrot', '#fashion', '#fall', '#chihuahuas', '#friends', '#travel', '#morning', '#wolf', '#sweden', '#monday']
top100['art'] = ['#art', '#artist', '#drawing', '#artwork', '#illustration', '#sketch', '#painting', '#draw', '#sketchbook', '#creative', '#design', '#love', '#photography', '#beautiful', '#pencil', '#ink', '#illustrator', '#drawings', '#portrait', '#arts', '#gallery', '#artistic', '#fashion', '#paint', '#doodle', '#nature', '#picture', '#watercolor', '#graphic', '#color', '#photo', '#sketching', '#music', '#style', '#pen', '#anime', '#cute', '#paintings', '#follow', '#paper', '#graphics', '#artists', '#like', '#travel', '#abstract', '#tattoo', '#cartoon', '#girl', '#masterpiece', '#sketches', '#beauty', '#inspiration', '#procreate', '#designer', '#comics', '#artworks', '#photographer', '#life', '#digital', '#illustrations', '#model', '#handmade', '#watercolour', '#draws', '#happy', '#painter', '#black', '#colorful', '#colors', '#comic', '#fun', '#landscape', '#exhibition', '#cool', '#colour', '#architecture', '#aesthetic', '#acrylic', '#smile', '#graffiti', '#arty', '#tattoos', '#amazing', '#lifestyle', '#sculpture', '#blue', '#rap', '#halloween', '#inked', '#funny', '#fantasy', '#character', '#me', '#summer', '#singer', '#studio', '#logo', '#collage', '#doodles', '#pictures']

for topic in top100.keys():
    top100[topic] = [r.replace('#', '') for r in top100[topic]]

In [5]:
top100_mapping = {}
for key in top100.keys():
    top100_stemmed = [ps.stem(word) for word in top100[key]]
    top100_mapping[key]={}
    for i in range(0,100):
        top100_mapping[key][top100_stemmed[i]]=top100[key][i]

In [6]:
def strip_punc(s):
    new_str = ""
    for c in s:
        if c in punctuation:
            new_str += " "
        else:
            new_str += c
    return new_str

In [7]:
# Preprocessing of input text

input_sentence = "Enjoy the little things in life Because someday you will look back and realize they were the BIG THINGS..."
# input_sentence = "Straight out of The Sound of Music scenery is what you can wake up to, hiking and mountain biking your days away and enjoying Tyrolean hospitality.⁠"
# input_sentence = "Magestic Mountains The views which i got during my roadtrip to Heaven of Earth Kasmir still give a pleasure to my soul @holidaycompass"
# input_sentence = "Have you tried these homemade Brownies yet? 🤤🍫 This brownie is ultra rich, fudgy, and chewy just like the kind you buy at the store but oh so much better with no preservatives! 🙌🏻📷: @handletheheat Follow us @recipesofholly & for recipes check out my profile"
# input_sentence = "#cutebabies #love_baby #kids #justbaby #angel #instagrambabies #ootd #pretty #instagramkids #bckcutie #cutekidsgotswag #myprettybabies #blackbabygoals #cutex10000 #beautifullymadebabies #bourgiebabies #gerberbaby #justbaby #mixedbabiesig #mixedracechildren #babylifezone #mixedkidzig #justbaby_world #newbornarrival #babypowergram #babiesarebeautiful #charmingkiddies #mixedbabiesworld #instadaily #cutebabies #instababy #babyboy #babies #mixedbabies #carters #babymodel #instafamous #igmodel"
# input_sentence = "#tigger #tiggertheyorkie #yorkie #yorkies #yorkiesofinstagram #dog #dogs #dogsofinstagram #dogstagram #pets #pet #petsofinsta #petsofinstagram #igmodel #explorepage #explore #myplayplace #cuddleseason #november #fallseason #fall"
# input_sentence = "#art #artist #drawing #jakeandamy #amysantiago #jakeperalta #sketch #draw #halloweenheist #traditionalart #sketchbook #sketching #creative #youngartist #artistsoninstagram #instaart #fanart #artsy #sketches #realistic #realism #realisticart #brooklynninenine #b99 #brooklyn99"
text = strip_punc(input_sentence.lower())
toks = word_tokenize(text)
toks_ = []
for tok in toks:
    tok = segment(tok)
    for t in tok:
        if wordnet.synsets(t):
            t = ps.stem(t)
            if t not in stop_words and len(t)>2:
                toks_.append(t)

input_words = toks_
print(input_words)

topic = "travel"
# topic = "food"
# topic = "baby"
# topic = "pets"
# topic = "art"

['enjoy', 'littl', 'thing', 'life', 'someday', 'look', 'back', 'realiz', 'big', 'thing']


In [8]:
# Preprocessing of input seed hashtag

input_sentence = "#life #enjoy #nevergiveup #bigthings #travel #travelblogger #worldtraveler #instagood #relax #fitnessmodel #beautiful #view #india #nature #instadaily #fit #modellife #king #waterfall #bodytransformation #bodypositive #cool #followforfollowback #follow"
# input_sentence = "#hiking #mountain #nature #beauty #adventure #travel #enjoy"
# input_sentence = "#biking #mountain #heaven #roadtrip #trip #tour #travelislife #holiday #love"
# input_sentence = "#foodporn #onthetable #eeeeeats #foodstyling #foodporn #igfood #healthyfood #food #foodie #foodphotography #foodpics #lovefood #healthybreakfast #fitfood #cleaneating #picoftheday #instagood #instayum #instarecipes #recipes #easyrecipes #foodies24hours #foodiepics #goodeating"
# input_sentence = "#cutebabies #love_baby #kids #justbaby #angel #instagrambabies #ootd #pretty #instagramkids #bckcutie #cutekidsgotswag #myprettybabies #blackbabygoals #cutex10000 #beautifullymadebabies #bourgiebabies #gerberbaby #justbaby #mixedbabiesig #mixedracechildren #babylifezone #mixedkidzig #justbaby_world #newbornarrival #babypowergram #babiesarebeautiful #charmingkiddies #mixedbabiesworld #instadaily #cutebabies #instababy #babyboy #babies #mixedbabies #carters #babymodel #instafamous #igmodel"
# input_sentence = "#tigger #tiggertheyorkie #yorkie #yorkies #yorkiesofinstagram #dog #dogs #dogsofinstagram #dogstagram #pets #pet #petsofinsta #petsofinstagram #igmodel #explorepage #explore #myplayplace #cuddleseason #november #fallseason #fall"
# input_sentence = "#art #artist #drawing #jakeandamy #amysantiago #jakeperalta #sketch #draw #halloweenheist #traditionalart #sketchbook #sketching #creative #youngartist #artistsoninstagram #instaart #fanart #artsy #sketches #realistic #realism #realisticart #brooklynninenine #b99 #brooklyn99"
text = strip_punc(input_sentence.lower())
toks = word_tokenize(text)
origs = []
for tok in toks:
    tok = segment(tok)
    for t in tok:
        if wordnet.synsets(t):
            t = ps.stem(t)
            if t not in stop_words and len(t)>2:
                origs.append(t)


print(origs)
# origs = set(origs)

['life', 'enjoy', 'never', 'give', 'big', 'thing', 'travel', 'travel', 'blogger', 'world', 'travel', 'good', 'relax', 'fit', 'model', 'beauti', 'view', 'india', 'natur', 'daili', 'fit', 'model', 'life', 'king', 'waterfal', 'bodi', 'transform', 'bodi', 'posit', 'cool', 'follow', 'follow']


In [9]:
model = gensim.models.Word2Vec.load("./models/word2vec."+topic)

In [10]:
print(len(list(model.wv.vocab.keys())))

4898


In [11]:
sent = []
for word in input_words:
    if word in model.wv.vocab:
        sent.append(model.wv[word])
    
print(len(sent))
sent = np.array(sent)
print(sent.shape)
embed = np.average(sent, axis = 0)
print(embed.shape)

9
(9, 300)
(300,)


In [12]:
# Finding similarity of top 100 hashtags using word2vec

ans = []
for hasht in top100_mapping[topic]:
    try:
        v1 = model.wv[hasht]
        simi = np.dot(v1, embed) / (np.linalg.norm(v1) * np.linalg.norm(embed))
        ans.append((simi, top100_mapping[topic][hasht]))
    except:
        pass

ans = sorted(ans, reverse = True)
print(ans[:20])

[(0.5388641, 'like'), (0.47820386, 'fun'), (0.40900812, 'life'), (0.3878241, 'smile'), (0.38648802, 'friends'), (0.3528942, 'cute'), (0.3525224, 'love'), (0.31312725, 'holidays'), (0.29667693, 'view'), (0.28741422, 'happiness'), (0.28185275, 'sightseeing'), (0.26759738, 'amazing'), (0.2652718, 'summer'), (0.26058048, 'fashion'), (0.2226773, 'beaches'), (0.21887957, 'trip'), (0.21510084, 'vacation'), (0.21314585, 'adventure'), (0.2088632, 'flowers'), (0.20397404, 'follow')]


In [13]:
k = 10
preds = [h[1] for h in ans[:k]]
print(preds)

['like', 'fun', 'life', 'smile', 'friends', 'cute', 'love', 'holidays', 'view', 'happiness']


In [14]:
print(preds)
origs = set(origs)
origs = list(origs)
print(origs)

['like', 'fun', 'life', 'smile', 'friends', 'cute', 'love', 'holidays', 'view', 'happiness']
['beauti', 'view', 'waterfal', 'travel', 'enjoy', 'transform', 'india', 'never', 'cool', 'daili', 'good', 'life', 'natur', 'give', 'thing', 'big', 'follow', 'world', 'blogger', 'bodi', 'fit', 'king', 'model', 'posit', 'relax']


In [15]:
# origs = ['life', 'enjoy', 'travel', 'relax', 'beauti', 'view', 'india', 'natur', 'fit', 'king', 'waterfal', 'cool', 'follow']
# preds = ['memories', 'friends', 'happiness', 'smile', 'fun', 'love', 'life', 'cute', 'amazing', 'like']

matr = np.empty((len(preds), len(origs)))

for i, pred in enumerate(preds):
    for j, orig in enumerate(origs):
        try:
            similarity = model.wv.similarity(ps.stem(orig), ps.stem(pred))
        except:
            similarity = 0
        matr[i][j] = similarity

In [16]:
# print(matr)
accs = []
tabl = PrettyTable()

tabl.field_names = ['Predicted'] + origs
for ind, row in enumerate(matr):
    l = [preds[ind]] + [round(i,2) for i in row]
    tabl.add_row(l)
    accs.append(max(row))
    indi = np.argmax(row)
    print(preds[ind] + "-------------" + origs[indi])
print(tabl)
# print(accs)
# print(np.mean(accs))

ans = 0
for acc in accs:
    if round(acc,1) >= 0.4:
        ans += 1
print(ans / len(preds))

like-------------follow
fun-------------enjoy
life-------------life
smile-------------thing
friends-------------thing
cute-------------cool
love-------------good
holidays-------------good
view-------------view
happiness-------------daili
+-----------+--------+------+----------+--------+-------+-----------+-------+-------+------+-------+------+------+-------+-------+-------+------+--------+-------+---------+-------+------+-------+-------+-------+-------+
| Predicted | beauti | view | waterfal | travel | enjoy | transform | india | never | cool | daili | good | life | natur |  give | thing | big  | follow | world | blogger |  bodi | fit  |  king | model | posit | relax |
+-----------+--------+------+----------+--------+-------+-----------+-------+-------+------+-------+------+------+-------+-------+-------+------+--------+-------+---------+-------+------+-------+-------+-------+-------+
|    like   |  0.28  | 0.29 |   0.0    |  0.26  |  0.33 |    0.06   |  0.22 |  0.3  | 0.33 |  0.41 | 0

In [17]:
top100 = {}
top100['travel'] = ['#travel', '#wanderlust', '#nature', '#travelling', '#traveling', '#traveller', '#photography', '#traveler', '#trip', '#travels', '#vacation', '#love', '#travelers', '#adventure', '#tourist', '#landscape', '#travellers', '#explore', '#holiday', '#beautiful', '#tourism', '#hiking', '#beach', '#photo', '#sunset', '#photographer', '#mountains', '#globetrotter', '#summer', '#art', '#sky', '#treking', '#europe', '#view', '#architecture', '#sea', '#happy', '#fun', '#city', '#sun', '#lifestyle', '#amazing', '#wanderer', '#italy', '#follow', '#backpacking', '#life', '#visiting', '#fashion', '#autumn', '#explorer', '#ocean', '#outdoors', '#india', '#world', '#mountain', '#beauty', '#spain', '#backpacker', '#style', '#like', '#france', '#exploring', '#trekking', '#clouds', '#asia', '#me', '#friends', '#usa', '#canon', '#happiness', '#blogger', '#holidays', '#ig', '#sunrise', '#smile', '#germany', '#girl', '#island', '#wander', '#paradise', '#turkey', '#discover', '#italia', '#voyage', '#flowers', '#landscapes', '#sightseeing', '#outdoor', '#history', '#indonesia', '#cute', '#forest', '#paris', '#food', '#australia', '#bali', '#pic', '#beaches', '#inspiration']
top100['food'] = ['#foodie', '#food', '#delicious', '#yummy', '#foodies', '#foods', '#eat', '#breakfast', '#dinner', '#tasty', '#cooking', '#lunch', '#homemade', '#dessert', '#love', '#eating', '#healthy', '#sweet', '#restaurant', '#photography', '#hungry', '#chef', '#blogger', '#travel', '#chocolate', '#baking', '#follow', '#cake', '#vegan', '#fresh', '#chicken', '#like', '#cook', '#amazing', '#blog', '#happy', '#favorite', '#brunch', '#coffee', '#weekend', '#fit', '#vegetarian', '#pasta', '#beautiful', '#pastry', '#fitness', '#gourmet', '#morning', '#desserts', '#seafood', '#eater', '#lifestyle', '#sweets', '#cafe', '#recipes', '#icecream', '#pizza', '#italy', '#culinary', '#candy', '#cheese', '#photographer', '#photo', '#cakes', '#recipe', '#noodles', '#diet', '#eats', '#rice', '#friends', '#cuisine', '#porridge', '#salad', '#nutrition', '#india', '#indonesia', '#bread', '#mornings', '#drinks', '#art', '#life', '#fun', '#gastronomy', '#sunday', '#cookies', '#kitchen', '#gym', '#me', '#beef', '#sushi', '#cupcakes', '#bake', '#spicy', '#saturday', '#fish', '#catering', '#burger', '#snack', '#music', '#delhi']
top100['baby'] = ['#baby', '#kids', '#love', '#babies', '#cute', '#family', '#mom', '#fashion', '#newborn', '#children', '#happy', '#beautiful', '#photography', '#girl', '#babys', '#adorable', '#sweet', '#motherhood', '#pregnancy', '#handmade', '#funny', '#lovely', '#smile', '#mommy', '#life', '#daughter', '#style', '#follow', '#sweetheart', '#like', '#pregnant', '#beauty', '#precious', '#kid', '#child', '#fun', '#girls', '#mummy', '#tiny', '#twins', '#amazing', '#toddler', '#model', '#photo', '#mama', '#toys', '#boy', '#sleep', '#matching', '#autumn', '#enjoy', '#friends', '#pretty', '#art', '#parenting', '#live', '#mother', '#me', '#amor', '#photographer', '#princess', '#fairy', '#sale', '#happiness', '#boys', '#boutique', '#lifestyle', '#portrait', '#fall', '#son', '#pink', '#nature', '#childhood', '#cuteness', '#maternity', '#cool', '#travel', '#angel', '#best', '#halloween', '#play', '#weekend', '#october', '#dad', '#innocent', '#brand', '#makeup', '#home', '#parenthood', '#nice', '#little', '#sunday', '#flowers', '#mum', '#canon', '#infant', '#party', '#wedding', '#summer', '#hot']
top100['jewellery'] = ['#jewellery', '#jewelry', '#fashion', '#accessories', '#earrings', '#necklace', '#style', '#jewels', '#handmade', '#beautiful', '#gemstone', '#trendy', '#gold', '#jewel', '#bracelet', '#rings', '#love', '#gems', '#silver', '#bracelets', '#design', '#ring', '#gemstones', '#crystals', '#shopping', '#stylish', '#art', '#pendant', '#diamond', '#cute', '#luxury', '#necklaces', '#bling', '#india', '#watches', '#handcrafted', '#designer', '#bijoux', '#trending', '#earring', '#bangles', '#bride', '#wedding', '#beauty', '#stone', '#tourmaline', '#charm', '#traditional', '#hyderabad', '#wholesaler', '#diamonds', '#bridal', '#photography', '#accessory', '#cabochons', '#tourmalines', '#stones', '#gem', '#goldplated', '#semiprecious', '#mumbai', '#silversmith', '#pendents', '#k', '#indian', '#sapphire', '#chennai', '#jewellers', '#girls', '#sale', '#jeweller', '#delhi', '#follow', '#fashionable', '#gift', '#bangalore', '#women', '#london', '#unique', '#ethnic', '#rawalpindi', '#crystal', '#vintage', '#oxidised', '#emerald', '#dubai', '#tucson', '#peridot', '#beads', '#makeup', '#saree', '#kunzite', '#chic', '#firecracker', '#pearl', '#lahore', '#choker', '#islamabad', '#wholesale', '#combo']
# top100['selfie'] = ['#me', '#love', '#follow', '#smile', '#photography', '#style', '#beautiful', '#fashion', '#happy', '#cute', '#fun', '#girl', '#life', '#portrait', '#like', '#pretty', '#photo', '#handsome', '#travel', '#nature', '#eyes', '#face', '#hair', '#summer', '#art', '#friends', '#amazing', '#model', '#food', '#beauty', '#pic', '#daily', '#girls', '#lifestyle', '#look', '#makeup', '#swag', '#photographer', '#fitness', '#autumn', '#cool', '#family', '#sky', '#picture', '#likes', '#sun', '#mood', '#music', '#followers', '#outfit', '#boy', '#colorful', '#self', '#motivation', '#lovely', '#gay', '#pose', '#landscape', '#black', '#inspiration', '#happiness', '#halloween', '#artist', '#gym', '#liker', '#pink', '#portraits', '#blogger', '#fit', '#sunset', '#wanderlust', '#design', '#funny', '#italy', '#traveling', '#hot', '#awesome', '#holiday', '#following', '#photos', '#india', '#night', '#blue', '#weekend', '#view', '#october', '#party', '#trip', '#red', '#tattoo', '#man', '#looks', '#stylish', '#sea', '#nice', '#good', '#beard', '#beach', '#woman', '#aesthetic']
top100['pet'] = ['#pet', '#pets', '#dog', '#cute', '#animals', '#love', '#dogs', '#cat', '#puppy', '#animal', '#cats', '#kitten', '#nature', '#kitty', '#meow', '#adorable', '#puppies', '#kittens', '#doggy', '#doggo', '#pup', '#photography', '#happy', '#funny', '#hound', '#eyes', '#beautiful', '#bunny', '#poodle', '#chihuahua', '#follow', '#rabbit', '#fluffy', '#paws', '#family', '#photo', '#lovely', '#chat', '#furry', '#daily', '#pomeranian', '#labrador', '#fun', '#autumn', '#sweet', '#maltese', '#smile', '#life', '#art', '#bunnies', '#hamster', '#baby', '#bird', '#me', '#world', '#bulldog', '#like', '#sleeping', '#tuesday', '#summer', '#parrots', '#husky', '#amor', '#rabbits', '#tot', '#zoo', '#doggie', '#meme', '#pug', '#istanbul', '#wildlife', '#beauty', '#adopt', '#girl', '#meowed', '#birds', '#purr', '#rescue', '#cavy', '#japan', '#doggies', '#halloween', '#photographer', '#woof', '#playtime', '#nice', '#portrait', '#relax', '#parakeet', '#singapore', '#parrot', '#fashion', '#fall', '#chihuahuas', '#friends', '#travel', '#morning', '#wolf', '#sweden', '#monday']
top100['art'] = ['#art', '#artist', '#drawing', '#artwork', '#illustration', '#sketch', '#painting', '#draw', '#sketchbook', '#creative', '#design', '#love', '#photography', '#beautiful', '#pencil', '#ink', '#illustrator', '#drawings', '#portrait', '#arts', '#gallery', '#artistic', '#fashion', '#paint', '#doodle', '#nature', '#picture', '#watercolor', '#graphic', '#color', '#photo', '#sketching', '#music', '#style', '#pen', '#anime', '#cute', '#paintings', '#follow', '#paper', '#graphics', '#artists', '#like', '#travel', '#abstract', '#tattoo', '#cartoon', '#girl', '#masterpiece', '#sketches', '#beauty', '#inspiration', '#procreate', '#designer', '#comics', '#artworks', '#photographer', '#life', '#digital', '#illustrations', '#model', '#handmade', '#watercolour', '#draws', '#happy', '#painter', '#black', '#colorful', '#colors', '#comic', '#fun', '#landscape', '#exhibition', '#cool', '#colour', '#architecture', '#aesthetic', '#acrylic', '#smile', '#graffiti', '#arty', '#tattoos', '#amazing', '#lifestyle', '#sculpture', '#blue', '#rap', '#halloween', '#inked', '#funny', '#fantasy', '#character', '#me', '#summer', '#singer', '#studio', '#logo', '#collage', '#doodles', '#pictures']
top100['architecture'] = ['#architecture', '#design', '#art', '#photography', '#building', '#travel', '#architect', '#city', '#buildings', '#urban', '#beautiful', '#style', '#minimal', '#street', '#architectural', '#interior', '#skyscraper', '#perspective', '#abstract', '#lines', '#town', '#architectures', '#geometric', '#designer', '#composition', '#geometry', '#arts', '#cities', '#home', '#landscape', '#nature', '#photo', '#photographer', '#interiors', '#house', '#italy', '#love', '#cityscape', '#modern', '#decor', '#pattern', '#luxury', '#sky', '#construction', '#architects', '#inspiration', '#arch', '#traveling', '#minimalism', '#render', '#wanderlust', '#sketch', '#rendering', '#europe', '#lifestyle', '#history', '#autumn', '#view', '#decoration', '#light', '#trip', '#amazing', '#concrete', '#paris', '#sunset', '#facade', '#artist', '#modernism', '#contemporary', '#sun', '#london', '#travelling', '#furniture', '#ig', '#life', '#france', '#church', '#traveller', '#canon', '#pic', '#explore', '#picture', '#details', '#colors', '#traveler', '#vacation', '#tourism', '#italia', '#milano', '#clouds', '#engineering', '#fashion', '#germany', '#spain', '#villa', '#wood', '#3d', '#exterior', '#blue', '#artwork']
top100['nature'] = ['#nature', '#photography', '#landscape', '#travel', '#beautiful', '#love', '#autumn', '#photo', '#sky', '#photographer', '#mountains', '#sunset', '#outdoors', '#clouds', '#art', '#flowers', '#beauty', '#tree', '#wanderlust', '#follow', '#amazing', '#adventure', '#trees', '#forest', '#view', '#explore', '#green', '#summer', '#life', '#sun', '#hiking', '#colors', '#wildlife', '#mountain', '#travelling', '#fashion', '#like', '#canon', '#trip', '#sea', '#ig', '#happy', '#water', '#natural', '#fall', '#outdoor', '#beach', '#style', '#traveling', '#fun', '#blue', '#landscapes', '#sunrise', '#flower', '#animals', '#lake', '#cute', '#picture', '#macro', '#me', '#river', '#earth', '#natures', '#friends', '#leaves', '#followers', '#vacation', '#plants', '#weather', '#smile', '#garden', '#india', '#girl', '#pic', '#november', '#photos', '#world', '#wild', '#birds', '#model', '#traveller', '#peace', '#lifestyle', '#bird', '#walk', '#animal', '#holiday', '#discover', '#trekking', '#portrait', '#morning', '#nice', '#sunday', '#ocean', '#sunsets', '#traveler', '#fitness', '#rain', '#italy', '#winter']

for topic in top100.keys():
    top100[topic] = [r.replace('#', '') for r in top100[topic]]

In [18]:
top100_mapping = {}
for key in top100.keys():
    top100_stemmed = [ps.stem(word) for word in top100[key]]
    top100_mapping[key]={}
    for i in range(0,100):
        top100_mapping[key][top100_stemmed[i]]=top100[key][i]

In [19]:
glob_corpora = []

In [20]:
final_dic = {}
for topic in top100.keys():
    final_dic[topic] = {}

In [21]:
for topic in top100.keys():
    for r, d, f in os.walk("./jsons/"+topic):
        print(topic)
        df = pd.read_csv('./csv/train/final_train_'+topic+".csv")
        dic = dict()
        for file in f:
            path = r+"/"+file
            print(path)
            if '.json' in file:
                file_ptr = open(path, "r")
                tmp_dic = json.load(file_ptr)
                dic.update(tmp_dic)
        final_dic[topic] = dic

travel
./jsons/travel/Luxarytravel.json
./jsons/travel/travelbook.json
./jsons/travel/Traveldeeper.json
./jsons/travel/Hiking.json
./jsons/travel/travelquotes.json
./jsons/travel/travelstoke.json
./jsons/travel/travel.json
./jsons/travel/traveladdict.json
./jsons/travel/travellersnotebook.json
./jsons/travel/travelguide.json
./jsons/travel/Travelabout.json
./jsons/travel/trip.json
./jsons/travel/solotravel.json
./jsons/travel/Travelphotography.json
./jsons/travel/travelbug.json
./jsons/travel/travelpic.json
./jsons/travel/travelgram.json
./jsons/travel/tourist.json
./jsons/travel/travelislife.json
./jsons/travel/beachvibes.json
./jsons/travel/treking.json
./jsons/travel/Travelawesome.json
./jsons/travel/traveltheglobe.json
./jsons/travel/travelworld.json
./jsons/travel/worldtraveller.json
./jsons/travel/Travellove.json
./jsons/travel/nature.json
./jsons/travel/travelcaptures.json
./jsons/travel/Citytravel.json
food
./jsons/food/foodphotography.json
./jsons/food/foodporn.json
./jsons/fo

In [22]:
model = {}
for topic in top100.keys():
    model[topic] = gensim.models.Word2Vec.load("./models/word2vec."+topic)

In [23]:
def extract_hashtag(text):
    hashtags = []
    others = ""
    texts = text.split(" ")
    for t in texts:
        if t=="":
            continue
        if t[0]=='#':
            hashtags.append(t)
        else:
            others = others + t + " "
    return hashtags,others

In [24]:
fans=0
c = 0
x=0
x1=0
# df = pd.read_csv('./csv/test/final_test.csv')
df = pd.read_csv('./csv/test/test-pred.csv')
baseline = True
for index,row in df.iterrows():
    
#     post_id = row['post_id'].split('.jpeg')[0]
#     topic = row['topic']
    post_id = row['filename'].split('.jpeg')[0]
    topic = row['class1']
    
    try:
        input_sentence = final_dic[topic][post_id]['text_des']
    except:
        x+=1
        continue
    
    hashtags,others = extract_hashtag(input_sentence)
    if(len(hashtags)<=3):
        x1+=1
        continue
    others = others + hashtags[0]
    others = others + " " + hashtags[1]
    others = others + " " + hashtags[2]
    del hashtags[0]
    del hashtags[0]
    del hashtags[0]
    
    text = strip_punc(others.lower())
    toks = word_tokenize(text)
    toks_ = []
    for tok in toks:
        tok = segment(tok)
        for t in tok:
            if wordnet.synsets(t):
                t = ps.stem(t)
                if t not in stop_words and len(t)>2:
                    toks_.append(t)

    input_words = toks_
    
    input_words = set(input_words)
    input_words = list(input_words)
    if(len(input_words)<=3):
        continue
    sent = []
    for word in input_words:
        if word in model[topic].wv.vocab:
            sent.append(model[topic].wv[word])

    sent = np.array(sent)
    embed = np.average(sent, axis = 0)
    
    ans = []
    for hasht in top100_mapping[topic]:
        try:
            v1 = model[topic].wv[hasht]
            simi = np.dot(v1, embed) / (np.linalg.norm(v1) * np.linalg.norm(embed))
            ans.append((simi, top100_mapping[topic][hasht]))
        except:
            pass

    ans = sorted(ans, reverse = True)
#     print(ans)
    k = 10
    preds = [h[1] for h in ans[:k]]
    
    tt = ""
    for h in hashtags:
        tt = tt+h
    text = strip_punc(tt.lower())
    toks = word_tokenize(text)
    toks_ = []
    for tok in toks:
        tok = segment(tok)
        for t in tok:
            if wordnet.synsets(t):
                t = ps.stem(t)
                if t not in stop_words and len(t)>2:
                    toks_.append(t)
    
    toks_ = set(toks_)
    toks_ = list(toks_)
    matr = np.empty((len(preds), len(toks_)))

    for i, pred in enumerate(preds):
        for j, orig in enumerate(toks_):
            try:
                v1 = model[topic].wv[ps.stem(pred)]
                v2 = model[topic].wv[ps.stem(orig)]
                similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            except:
                similarity = 0
            matr[i][j] = similarity
    
    
    accs = []
    tabl = PrettyTable()
    tabl.field_names = ['Predicted'] + toks_
    for ind, row in enumerate(matr):
        l = [preds[ind]] + [round(i,2) for i in row]
        tabl.add_row(l)
        try:
            accs.append(max(row))
        except:
            accs.append(0)
#         indi = np.argmax(row)

    acc_ans = 0
    for acc in accs:
        if round(acc,1) >= 0.4:
            acc_ans += 1

    acc_ans=float(acc_ans)/float(len(preds))
#     print(acc_ans)
    fans=fans+acc_ans
    c+=1

In [25]:
print(x)

865


In [26]:
print(x1)

431


In [27]:
print(c)

1471


In [28]:
print(len(df))

3000


In [29]:
print((fans/c)*100)

86.5465669612509
